# 残差网络 

In [1]:
import torch 
import torchvision
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import transforms 
from torch.utils.data import DataLoader,Dataset 

In [2]:
model = models.resnet18(pretrained=True)
model

D:\venvs\dl\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\venvs\dl\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
model.fc = nn.Linear(in_features=512,out_features=4) 
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 参数冻结与解冻 

In [4]:
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters(): 
    param.requires_grad = True

In [5]:
transforms_Train = transforms.Compose(
     [
        #数据增强 
        transforms.Resize(224),
        transforms.RandomCrop(192),
        transforms.RandomHorizontalFlip(), #随机水平翻转 
        transforms.RandomRotation(degrees=20), #随机旋转  
        transforms.ColorJitter(brightness=0.5), #随机扰动
        transforms.ColorJitter(contrast=0.5), #增加对比度
        # transforms.Resize((192,192)), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5]),
        
        
    ]
)

In [6]:
transforms_Tst = transforms.Compose(
    [
        transforms.Resize((192,192)),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
    ]
)

In [7]:
Train_DS = torchvision.datasets.ImageFolder(
    '../01计算机视觉基础/4weather/train', 
    transform = transforms_Train
)
Tst_DS = torchvision.datasets.ImageFolder(
    '../01计算机视觉基础/4weather/test/', 
    transform=transforms_Tst
)

In [8]:
Train_DL = DataLoader(Train_DS,batch_size=32,shuffle=True)
Tst_DL = DataLoader(Tst_DS,batch_size=64,shuffle=False)

In [9]:
#设置优化器 ,因为我们已经冻结了原来的层,这时候只需要关注最后一层就可以了
optimizer = torch.optim.Adam(model.fc.parameters(),lr=0.0001) #学习率不要太高,因为已经训练好了
#设置损失函数 
loss_fn = nn.CrossEntropyLoss()

In [10]:
def fit_gpu(Train_DL,TST_DL,Model_m,epoch_,optim,loss_fn): 
    correct = 0
    total = 0
    running_loss = 0
    tstcorrect = 0
    tsttotal = 0
    tstrunning_loss = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Model_m.to(device)
  
    Model_m.train()  # 设置为训练模式,此时dropout层会发挥作用
    for x, y in Train_DL:
        x = x.to(device)
        y = y.to(device)
        
        y_pred = Model_m(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()

        # 每个epoch结束后评估模型
        # 所有 batch 都训练完后，再计算整个 epoch 的准确率和损失
        with torch.no_grad():
            y_pred = torch.argmax(y_pred,dim=1)  #沿着类别维度找到最大值的索引位置
            correct += (y_pred == y).sum().item() #预测正确的个数
            # 因为(y_pred == y).sum()是一个张量,所以为了等式可以数值操作,我们用.item()取值
            total += y.size(0) #样本的个数,也就是size返回值的第一个返回值,即行数
            running_loss += loss.item()

        
    epoch_acc = correct/total
    epoch_loss = running_loss/len(Train_DL.dataset)

    #测试阶段 
    Model_m.eval() #转为eval()模式,此时dropout层不会起作用
    with torch.no_grad(): #测试不需要反向传播,而是用训练好的模型来测试测试集的数据
         for x, y in TST_DL:
             x, y = x.to(device), y.to(device)
             y_pred =Model_m(x)
             loss = loss_fn(y_pred, y)
             
             y_pred = torch.argmax(y_pred,dim=1)   #获取真正的预测结果,不懂就往上巴拉argmax
             tstcorrect += (y_pred == y).sum().item() #预测正确的个数
             tsttotal += y.size(0) #样本的个数
             tstrunning_loss += loss.item()
             
    tstepoch_acc = tstcorrect/tsttotal
    tstepoch_loss = tstrunning_loss/len(TST_DL.dataset)

    print(
        f'epoch:{epoch} | loss:{epoch_loss:.3f} | acc:{epoch_acc:.3f} | tstloss:{tstepoch_loss:.3f} | tstacc:{tstepoch_acc:.3f}'
    )

    return epoch_loss,epoch_acc,tstepoch_loss,tstepoch_acc


In [11]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[-6.7192e-03, -1.4244e-02,  2.4399e-02,  ..., -3.3374e-02,
            -1.4130e-02,  2.7595e-02],
           [-2.7468e-02,  9.3642e-03,  5.9742e-03,  ..., -9.5583e-03,
             3.4777e-02, -2.8015e-02],
           [ 2.3504e-02,  1.4099e-02,  1.2068e-02,  ...,  2.8718e-03,
             1.3207e-02,  2.0251e-04],
           [ 3.1553e-02,  4.3489e-05,  1.7658e-02,  ...,  9.2012e-03,
             2.6336e-02,  1.2223e-02]], requires_grad=True),
   Parameter containing:
   tensor([ 0.0073, -0.0101,  0.0325, -0.0055], requires_grad=True)],
  'lr': 0.0001,
  'betas': (0.9, 0.999),
  'eps': 1e-08,
  'weight_decay': 0,
  'amsgrad': False,
  'maximize': False,
  'foreach': None,
  'capturable': False,
  'differentiable': False,
  'fused': None,
  'decoupled_weight_decay': False}]

In [12]:

from torch.optim.lr_scheduler import StepLR

scheduler = StepLR(optimizer=optimizer, step_size=5, gamma=0.9) #设置指定的优化器衰减策略,每5个epoch衰减一次,指定每次的被乘因子为0.9
train_loss_gpu = []
train_acc_gpu = []
tst_loss_gpu = []
tst_acc_gpu = []
epochs = 10

for epoch in range(epochs):
      # 每个 epoch 都训练
    epoch_loss, epoch_acc, tstepoch_loss, tstepoch_acc = fit_gpu(
        Train_DL=Train_DL,
        TST_DL=Tst_DL,
        Model_m=model,
        epoch_=epoch,
        optim=optimizer,
        loss_fn=loss_fn
    )
    
    # 记录指标
    train_loss_gpu.append(epoch_loss)
    train_acc_gpu.append(epoch_acc)
    tst_loss_gpu.append(tstepoch_loss)
    tst_acc_gpu.append(tstepoch_acc)
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}: 学习率已更新为 {current_lr:.2e}")

epoch:0 | loss:0.048 | acc:0.258 | tstloss:0.024 | tstacc:0.351
Epoch 1: 学习率已更新为 1.00e-04
epoch:1 | loss:0.044 | acc:0.313 | tstloss:0.022 | tstacc:0.453
Epoch 2: 学习率已更新为 1.00e-04
epoch:2 | loss:0.041 | acc:0.390 | tstloss:0.021 | tstacc:0.489
Epoch 3: 学习率已更新为 1.00e-04
epoch:3 | loss:0.038 | acc:0.516 | tstloss:0.020 | tstacc:0.538
Epoch 4: 学习率已更新为 1.00e-04
epoch:4 | loss:0.035 | acc:0.564 | tstloss:0.018 | tstacc:0.613
Epoch 5: 学习率已更新为 9.00e-05
epoch:5 | loss:0.034 | acc:0.613 | tstloss:0.018 | tstacc:0.596
Epoch 6: 学习率已更新为 9.00e-05
epoch:6 | loss:0.032 | acc:0.698 | tstloss:0.017 | tstacc:0.640
Epoch 7: 学习率已更新为 9.00e-05
epoch:7 | loss:0.031 | acc:0.691 | tstloss:0.016 | tstacc:0.680
Epoch 8: 学习率已更新为 9.00e-05
epoch:8 | loss:0.029 | acc:0.769 | tstloss:0.015 | tstacc:0.711
Epoch 9: 学习率已更新为 9.00e-05
epoch:9 | loss:0.029 | acc:0.748 | tstloss:0.015 | tstacc:0.711
Epoch 10: 学习率已更新为 8.10e-05
